In [14]:
import os
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split, Subset
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn as nn
import optuna
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import copy
import time
import logging
from typing import Any, Optional, List, Tuple, Dict
from torch.utils.data import TensorDataset, DataLoader, Subset, WeightedRandomSampler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

定义准确率函数

In [15]:
def accuracy(outputs: torch.Tensor, labels: torch.Tensor) -> float:
    """
    计算准确率。

    参数:
    - outputs (torch.Tensor): 模型输出的 logits。
    - labels (torch.Tensor): 真实标签。

    返回:
    - float: 准确率。
    """
    _, preds = torch.max(outputs, 1)
    return (preds == labels).float().mean().item()

集成模型（基于F1分数分配权重）

In [16]:
class EnsembleModel(nn.Module):
    """集成模型类，使用F1分数优化权重"""
    def __init__(self, base_models: List[nn.Module], ensemble_weights: Optional[List[float]] = None):
        super(EnsembleModel, self).__init__()
        self.base_models = nn.ModuleList(base_models)
        
        if ensemble_weights is None:
            # 如果没有提供权重，使用均等权重
            self.ensemble_weights = torch.ones(len(base_models)) / len(base_models)
        else:
            # 如果提供了权重（可能是F1分数），归一化处理
            weights = torch.tensor(ensemble_weights)
            self.ensemble_weights = weights / weights.sum()
        
        # 记录权重分配
        logging.info(f"集成模型权重: {self.ensemble_weights.tolist()}")
        
        self.register_buffer('weights', self.ensemble_weights)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        predictions = []
        for model in self.base_models:
            with torch.no_grad():
                model.eval()
                pred = model(x)
                predictions.append(torch.softmax(pred, dim=1))
        
        stacked_preds = torch.stack(predictions, dim=0)
        weighted_preds = stacked_preds * self.weights.view(-1, 1, 1)
        return weighted_preds.sum(dim=0)

Transformer模型框架+BN层

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class PositionalEncoding(nn.Module):
    """位置编码模块"""
    
    def __init__(self, d_model: int, max_seq_length: int = 5000, dropout: float = 0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # 确保 d_model 是偶数
        d_model = (d_model // 2) * 2
        
        # 创建位置编码矩阵
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        # 计算正弦和余弦位置编码
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # 调整形状以匹配输入 [1, max_seq_len, d_model]
        
        # 注册为缓冲区，使其不会被视为模型参数
        self.register_buffer('pe', pe)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        添加位置编码到输入张量
        
        Args:
            x: 输入张量 [batch_size, seq_len, embedding_dim]
        Returns:
            添加位置编码后的张量
        """
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_prob=0.3, 
                 num_heads=8, dim_feedforward=2048, num_encoder_layers=2, max_seq_length=5000):
        """
        Transformer 模型框架（带 BN 层 + 位置编码）
        
        Args:
            input_size (int): 输入特征维度
            hidden_sizes (List[int]): 隐藏层维度（未使用，Transformer 主要靠 Self-Attention）
            output_size (int): 输出维度
            dropout_prob (float): Dropout 概率
            num_heads (int): Multi-Head Attention 头数
            dim_feedforward (int): Transformer 前馈层维度
            num_encoder_layers (int): Transformer 编码层数
            max_seq_length (int): 位置编码的最大序列长度
        """
        super(TransformerModel, self).__init__()

        # ✅ 输入层：线性层 + BN
        self.input_layer = nn.Linear(input_size, dim_feedforward)
        self.input_bn = nn.BatchNorm1d(dim_feedforward)

        # ✅ 位置编码模块
        self.positional_encoding = PositionalEncoding(dim_feedforward, max_seq_length, dropout_prob)

        # ✅ Transformer 编码层
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=dim_feedforward, 
            nhead=num_heads, 
            dim_feedforward=dim_feedforward, 
            dropout=dropout_prob, 
            batch_first=True  # (batch, seq, feature)
        )
        
        # ✅ 多层 Transformer Encoder
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        
        # ✅ 输出层：全连接 + BN
        self.output_layer = nn.Linear(dim_feedforward, output_size)
        self.output_bn = nn.BatchNorm1d(output_size)
        
    def forward(self, x):
        """
        前向传播过程
        
        Args:
            x (torch.Tensor): 输入张量，形状为 [batch_size, seq_length, input_size]
        
        Returns:
            torch.Tensor: 输出张量，形状为 [batch_size, output_size]
        """
        batch_size, seq_length, _ = x.shape
        
        # ✅ 线性变换 & BN
        x = self.input_layer(x)
        x = self.input_bn(x.view(-1, x.shape[-1])).view(batch_size, seq_length, -1)
        x = F.relu(x)
        
        # ✅ 添加位置编码
        x = self.positional_encoding(x)
        
        # ✅ Transformer 编码层
        x = self.transformer_encoder(x)  # [batch, seq, dim_feedforward]
        
        # ✅ 池化：取 Transformer 输出的平均值作为全局特征
        x = torch.mean(x, dim=1)  # [batch, dim_feedforward]
        
        # ✅ 输出层
        x = self.output_layer(x)
        x = self.output_bn(x)
        
        return x


定义训练与评估循环函数

In [18]:
def train_eval_transformer(
    model: nn.Module,
    train_loader: DataLoader, 
    val_loader: DataLoader,
    loss_fn: nn.Module,
    optimizer: torch.optim.Optimizer,
    scheduler: lr_scheduler._LRScheduler,
    epochs: int,
    device: torch.device,
    log_epoch: Optional[List[str]] = None,
    max_grad_norm: Optional[float] = None
) -> Tuple[float, float, float, float, np.ndarray, np.ndarray, np.ndarray, np.ndarray, Dict]:
    """
    训练与评估Transformer模型的主循环函数，支持类别平衡评估，使用F1分数选择最佳模型。
    增加了精确率、召回率、特异性和MCC评估指标。
    
    Args:
        model: 神经网络模型
        train_loader: 训练数据加载器
        val_loader: 验证数据加载器 
        loss_fn: 损失函数
        optimizer: 优化器
        scheduler: 学习率调度器
        epochs: 训练轮数
        device: 训练设备
        log_epoch: 可选的日志记录列表
        max_grad_norm: 可选的梯度裁剪阈值

    Returns:
        Tuple[float, float, float, float, np.ndarray, np.ndarray, np.ndarray, np.ndarray, Dict]:
            - train_loss: 最终训练损失
            - val_loss: 最终验证损失
            - train_acc: 最终训练准确率
            - val_acc: 最终验证准确率
            - train_probs: 训练集预测概率
            - train_labels: 训练集真实标签
            - val_probs: 验证集预测概率
            - val_labels: 验证集真实标签
            - model_state: 最佳模型状态字典
    """
    # 导入指标计算函数
    from sklearn.metrics import f1_score, precision_score, recall_score, matthews_corrcoef, confusion_matrix
    
    # 初始化记录器
    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': [],
        'train_class_acc': [],     # 类别准确率记录
        'val_class_acc': [],       # 类别准确率记录
        'train_f1': [],            # F1分数记录
        'val_f1': [],              # F1分数记录
        'train_precision': [],     # 精确率记录
        'val_precision': [],       # 精确率记录
        'train_recall': [],        # 召回率记录
        'val_recall': [],          # 召回率记录
        'train_specificity': [],   # 特异性记录
        'val_specificity': [],     # 特异性记录
        'train_mcc': [],           # MCC记录
        'val_mcc': [],             # MCC记录
        'train_norm_mcc': [],      # 归一化MCC记录
        'val_norm_mcc': []         # 归一化MCC记录
    }
    
    best_model_state = None
    best_val_f1 = float('-inf')  # 使用F1分数选择最佳模型
    
    # 检查是否使用了带权重的损失函数
    has_weights = hasattr(loss_fn, 'weight') and loss_fn.weight is not None
    if has_weights:
        logging.info(f"使用带权重的损失函数，权重: {loss_fn.weight.cpu().numpy()}")
    
    for epoch in range(epochs):
        epoch_start_time = time.time()
        
        # ===== 训练阶段 =====
        model.train()
        train_loss = 0.0
        train_acc = 0.0
        epoch_train_probs = []
        epoch_train_labels = []
        epoch_train_preds = []  # 存储预测类别
        num_train_batches = 0
        
        for inputs, labels in train_loader:
            if inputs.size(0) <= 1:  # 跳过大小为1的批次
                continue
                
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # 前向传播
            outputs = model(inputs)
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            
            # 计算损失和准确率
            loss = loss_fn(outputs, labels)
            batch_acc = accuracy(outputs, labels)
            
            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            
            # 梯度裁剪
            if max_grad_norm is not None:
                if isinstance(model, nn.DataParallel):
                    torch.nn.utils.clip_grad_norm_(
                        model.module.parameters(), 
                        max_grad_norm
                    )
                else:
                    torch.nn.utils.clip_grad_norm_(
                        model.parameters(), 
                        max_grad_norm
                    )
                
            optimizer.step()
            
            # 累积损失和准确率
            train_loss += loss.item()
            train_acc += batch_acc
            num_train_batches += 1
            
            # 保存预测概率、预测类别和标签
            probs = torch.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, dim=1)
            epoch_train_probs.append(probs.detach().cpu().numpy())
            epoch_train_preds.append(preds.cpu().numpy())
            epoch_train_labels.append(labels.cpu().numpy())
        
        # 计算训练阶段平均指标
        avg_train_loss = train_loss / num_train_batches
        avg_train_acc = train_acc / num_train_batches
        
        # 计算每个类别的准确率和其他指标
        train_preds_all = np.concatenate(epoch_train_preds)
        train_labels_all = np.concatenate(epoch_train_labels)
        
        # F1分数
        train_f1 = f1_score(train_labels_all, train_preds_all, average='weighted')
        
        # 精确率
        train_precision = precision_score(train_labels_all, train_preds_all, average='weighted')
        
        # 召回率
        train_recall = recall_score(train_labels_all, train_preds_all, average='weighted')
        
        # 特异性 (Specificity) - 对于二分类问题
        train_cm = confusion_matrix(train_labels_all, train_preds_all)
        train_tn = train_cm[0, 0]
        train_fp = train_cm[0, 1]
        train_specificity = train_tn / (train_tn + train_fp) if (train_tn + train_fp) > 0 else 0
        
        # MCC (Matthews Correlation Coefficient)
        train_mcc = matthews_corrcoef(train_labels_all, train_preds_all)
        
        # 归一化MCC到[0,1]区间
        train_norm_mcc = (train_mcc + 1) / 2
        
        # 计算每个类别的准确率
        train_class_counts = np.bincount(train_labels_all, minlength=2)
        train_class_correct = np.bincount(
            train_labels_all[train_preds_all == train_labels_all], 
            minlength=2
        )
        train_class_acc = np.zeros(2)  # 初始化为零，避免除以零的问题
        for i in range(2):
            if train_class_counts[i] > 0:
                train_class_acc[i] = train_class_correct[i] / train_class_counts[i]
        
        # 记录训练指标
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(avg_train_acc)
        history['train_class_acc'].append(train_class_acc.tolist())
        history['train_f1'].append(train_f1)
        history['train_precision'].append(train_precision)
        history['train_recall'].append(train_recall)
        history['train_specificity'].append(train_specificity)
        history['train_mcc'].append(train_mcc)
        history['train_norm_mcc'].append(train_norm_mcc)
        
        # ===== 验证阶段 =====
        model.eval()
        val_loss = 0.0
        val_acc = 0.0
        epoch_val_probs = []
        epoch_val_labels = []
        epoch_val_preds = []  # 存储预测类别
        num_val_batches = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                if inputs.size(0) <= 1:
                    continue
                    
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # 前向传播
                outputs = model(inputs)
                if isinstance(outputs, tuple):
                    outputs = outputs[0]
                
                # 计算损失和准确率
                loss = loss_fn(outputs, labels)
                batch_acc = accuracy(outputs, labels)
                
                val_loss += loss.item()
                val_acc += batch_acc
                num_val_batches += 1
                
                # 保存预测概率、预测类别和标签
                probs = torch.softmax(outputs, dim=1)
                _, preds = torch.max(outputs, dim=1)
                epoch_val_probs.append(probs.cpu().numpy())
                epoch_val_preds.append(preds.cpu().numpy())
                epoch_val_labels.append(labels.cpu().numpy())
        
        # 计算验证阶段平均指标
        avg_val_loss = val_loss / num_val_batches
        avg_val_acc = val_acc / num_val_batches
        
        # 计算每个类别的准确率和其他指标
        val_preds_all = np.concatenate(epoch_val_preds)
        val_labels_all = np.concatenate(epoch_val_labels)
        
        # F1分数
        val_f1 = f1_score(val_labels_all, val_preds_all, average='weighted')
        
        # 精确率
        val_precision = precision_score(val_labels_all, val_preds_all, average='weighted')
        
        # 召回率
        val_recall = recall_score(val_labels_all, val_preds_all, average='weighted')
        
        # 特异性 (Specificity) - 对于二分类问题
        val_cm = confusion_matrix(val_labels_all, val_preds_all)
        val_tn = val_cm[0, 0]
        val_fp = val_cm[0, 1]
        val_specificity = val_tn / (val_tn + val_fp) if (val_tn + val_fp) > 0 else 0
        
        # MCC (Matthews Correlation Coefficient)
        val_mcc = matthews_corrcoef(val_labels_all, val_preds_all)
        
        # 归一化MCC到[0,1]区间
        val_norm_mcc = (val_mcc + 1) / 2
        
        # 计算每个类别的准确率
        val_class_counts = np.bincount(val_labels_all, minlength=2)
        val_class_correct = np.bincount(
            val_labels_all[val_preds_all == val_labels_all], 
            minlength=2
        )
        val_class_acc = np.zeros(2)  # 初始化为零，避免除以零的问题
        for i in range(2):
            if val_class_counts[i] > 0:
                val_class_acc[i] = val_class_correct[i] / val_class_counts[i]
        
        # 记录验证指标
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(avg_val_acc)
        history['val_class_acc'].append(val_class_acc.tolist())
        history['val_f1'].append(val_f1)
        history['val_precision'].append(val_precision)
        history['val_recall'].append(val_recall)
        history['val_specificity'].append(val_specificity)
        history['val_mcc'].append(val_mcc)
        history['val_norm_mcc'].append(val_norm_mcc)
        
        # 更新最佳模型状态，使用F1分数作为指标
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            if isinstance(model, nn.DataParallel):
                best_model_state = copy.deepcopy(model.module.state_dict())
            else:
                best_model_state = copy.deepcopy(model.state_dict())
            logging.info(f"*** 新的最佳模型 (F1分数: {val_f1:.4f}) ***")
        
        # 更新学习率
        if isinstance(scheduler, lr_scheduler.ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
        else:
            scheduler.step()
            
        # 获取当前学习率
        current_lr = optimizer.param_groups[0]['lr']
        
        # 记录训练日志，包含所有新增的评估指标
        epoch_log = (
            f'轮次 {epoch+1}/{epochs}, '
            f'训练损失: {avg_train_loss:.4f}, 训练准确率: {avg_train_acc:.4f}, '
            f'类别0/1: {train_class_acc[0]:.4f}/{train_class_acc[1]:.4f}, '
            f'训练F1: {train_f1:.4f}, 训练精确率: {train_precision:.4f}, '
            f'训练召回率: {train_recall:.4f}, 训练特异性: {train_specificity:.4f}, '
            f'训练MCC: {train_mcc:.4f}, 训练归一化MCC: {train_norm_mcc:.4f}, '
            f'验证损失: {avg_val_loss:.4f}, 验证准确率: {avg_val_acc:.4f}, '
            f'类别0/1: {val_class_acc[0]:.4f}/{val_class_acc[1]:.4f}, '
            f'验证F1: {val_f1:.4f}, 验证精确率: {val_precision:.4f}, '
            f'验证召回率: {val_recall:.4f}, 验证特异性: {val_specificity:.4f}, '
            f'验证MCC: {val_mcc:.4f}, 验证归一化MCC: {val_norm_mcc:.4f}, '
            f'学习率: {current_lr:.6f}'
        )
        logging.info(epoch_log)
        logging.info(f"本轮用时: {time.time() - epoch_start_time:.2f} 秒")
        logging.info("——————————————————————————————")
        
        if log_epoch is not None:
            log_epoch.append(epoch_log)
    
    # 加载最佳模型状态
    if best_model_state is not None:
        if isinstance(model, nn.DataParallel):
            model.module.load_state_dict(best_model_state)
        else:
            model.load_state_dict(best_model_state)
    
    # 整理最终结果
    final_train_probs = np.concatenate([p for p in epoch_train_probs], axis=0)
    final_train_labels = np.concatenate([l for l in epoch_train_labels], axis=0)
    final_val_probs = np.concatenate([p for p in epoch_val_probs], axis=0)
    final_val_labels = np.concatenate([l for l in epoch_val_labels], axis=0)
    
    # 计算最终的类别准确率和所有评估指标
    final_train_preds = np.argmax(final_train_probs, axis=1)
    final_val_preds = np.argmax(final_val_probs, axis=1)
    
    # F1分数
    final_train_f1 = f1_score(final_train_labels, final_train_preds, average='weighted')
    final_val_f1 = f1_score(final_val_labels, final_val_preds, average='weighted')
    
    # 精确率
    final_train_precision = precision_score(final_train_labels, final_train_preds, average='weighted')
    final_val_precision = precision_score(final_val_labels, final_val_preds, average='weighted')
    
    # 召回率
    final_train_recall = recall_score(final_train_labels, final_train_preds, average='weighted')
    final_val_recall = recall_score(final_val_labels, final_val_preds, average='weighted')
    
    # 特异性
    final_train_cm = confusion_matrix(final_train_labels, final_train_preds)
    final_train_tn = final_train_cm[0, 0]
    final_train_fp = final_train_cm[0, 1]
    final_train_specificity = final_train_tn / (final_train_tn + final_train_fp) if (final_train_tn + final_train_fp) > 0 else 0
    
    final_val_cm = confusion_matrix(final_val_labels, final_val_preds)
    final_val_tn = final_val_cm[0, 0]
    final_val_fp = final_val_cm[0, 1]
    final_val_specificity = final_val_tn / (final_val_tn + final_val_fp) if (final_val_tn + final_val_fp) > 0 else 0
    
    # MCC
    final_train_mcc = matthews_corrcoef(final_train_labels, final_train_preds)
    final_val_mcc = matthews_corrcoef(final_val_labels, final_val_preds)
    
    # 归一化MCC
    final_train_norm_mcc = (final_train_mcc + 1) / 2
    final_val_norm_mcc = (final_val_mcc + 1) / 2
    
    # 类别准确率
    train_class_acc = np.zeros(2)
    val_class_acc = np.zeros(2)
    
    for i in range(2):
        train_class_mask = (final_train_labels == i)
        val_class_mask = (final_val_labels == i)
        
        if np.sum(train_class_mask) > 0:
            train_class_acc[i] = np.mean(final_train_preds[train_class_mask] == final_train_labels[train_class_mask])
        
        if np.sum(val_class_mask) > 0:
            val_class_acc[i] = np.mean(final_val_preds[val_class_mask] == final_val_labels[val_class_mask])
    
    # 输出最终各项指标
    logging.info(f"最终训练类别准确率 - 类别0: {train_class_acc[0]:.4f}, 类别1: {train_class_acc[1]:.4f}")
    logging.info(f"最终验证类别准确率 - 类别0: {val_class_acc[0]:.4f}, 类别1: {val_class_acc[1]:.4f}")
    logging.info(f"最终F1分数 - 训练: {final_train_f1:.4f}, 验证: {final_val_f1:.4f}")
    logging.info(f"最终精确率 - 训练: {final_train_precision:.4f}, 验证: {final_val_precision:.4f}")
    logging.info(f"最终召回率 - 训练: {final_train_recall:.4f}, 验证: {final_val_recall:.4f}")
    logging.info(f"最终特异性 - 训练: {final_train_specificity:.4f}, 验证: {final_val_specificity:.4f}")
    logging.info(f"最终MCC - 训练: {final_train_mcc:.4f}, 验证: {final_val_mcc:.4f}")
    logging.info(f"最终归一化MCC - 训练: {final_train_norm_mcc:.4f}, 验证: {final_val_norm_mcc:.4f}")
    
    return (
        history['train_loss'][-1],    # 最终训练损失
        history['val_loss'][-1],      # 最终验证损失
        history['train_acc'][-1],     # 最终训练准确率
        history['val_acc'][-1],       # 最终验证准确率
        final_train_probs,           # 训练集预测概率
        final_train_labels,          # 训练集真实标签
        final_val_probs,             # 验证集预测概率
        final_val_labels,            # 验证集真实标签
        best_model_state             # 最佳模型状态
    )

 Transformer 的 K-Fold 训练与评估函数

In [19]:
def kfold_train_eval_transformer(
    model_class: Any,
    dataset: TensorDataset,
    loss_fn: nn.Module,
    optimizer_class: Any,
    optimizer_kwargs: dict,
    scheduler_class: Any,
    scheduler_kwargs: dict,
    epochs: int,
    device: torch.device,
    device_ids: Optional[List[int]],
    num_folds: int = 5,
    output_dir: str = r"C:\Users\USTC\Desktop\地磁论文（5级以上）\对比模型结果\Transformer模型结果",
    model_name: str = "TransformerModel",
    window_name: str = "7天",
    mapped_window_name: str = "window-7",
    max_grad_norm: Optional[float] = None,
    input_size: int = 1,
    output_size: int = 2,
    hidden_sizes: List[int] = [512, 256],  # 修改为多个隐藏层大小的列表
    dropout_prob: float = 0.3,
    batch_size: int = 32,
    num_heads: int = 8,
    dim_feedforward: int = 2048,
    num_encoder_layers: int = 2
) -> Dict[str, List]:
    """
    使用分层K折交叉验证进行Transformer模型的训练和评估，使用F1分数优化模型选择。
    
    Args:
        model_class: 模型类
        dataset: 数据集
        loss_fn: 损失函数
        optimizer_class: 优化器类
        optimizer_kwargs: 优化器参数
        scheduler_class: 学习率调度器类
        scheduler_kwargs: 学习率调度器参数
        epochs: 训练轮数
        device: 训练设备
        device_ids: GPU设备ID列表
        num_folds: 交叉验证折数
        output_dir: 模型保存目录
        model_name: 模型名称
        window_name: 时间窗口名称
        mapped_window_name: 映射后的窗口名称
        max_grad_norm: 梯度裁剪阈值
        input_size: 输入维度
        output_size: 输出维度
        hidden_sizes: Transformer隐藏层大小列表，如 [512, 256]
        dropout_prob: Dropout概率
        batch_size: 批次大小
        num_heads: Transformer注意力头数量
        dim_feedforward: Transformer前馈网络维度
        num_encoder_layers: Transformer编码器层数
    
    Returns:
        Dict[str, List]: 包含训练结果的字典
    """
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import f1_score, precision_score, recall_score, matthews_corrcoef, confusion_matrix
    
    os.makedirs(output_dir, exist_ok=True)
    
    results_dict = {
        "train_losses": [],
        "val_losses": [],
        "train_accs": [],
        "val_accs": [],
        "train_f1s": [],       # F1分数记录
        "val_f1s": [],         # F1分数记录
        "train_precisions": [], # 精确率记录
        "val_precisions": [],   # 精确率记录
        "train_recalls": [],    # 召回率记录
        "val_recalls": [],      # 召回率记录
        "train_specificities": [], # 特异性记录
        "val_specificities": [],   # 特异性记录
        "train_mccs": [],      # MCC记录
        "val_mccs": [],        # MCC记录
        "train_norm_mccs": [], # 归一化MCC记录
        "val_norm_mccs": [],   # 归一化MCC记录
        "fold_results": []     # 存储每个折叠的详细结果
    }
    
    total_start_time = time.time()
    
    # 获取数据集的标签，用于分层划分
    all_data = dataset.tensors[0].numpy()
    all_labels = dataset.tensors[1].numpy()
    
    # 创建分层K折交叉验证
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
    
    num_gpus = len(device_ids) if device_ids else 1
    adjusted_batch_size = batch_size * num_gpus
    
    logging.info(f"使用设备数量: {num_gpus}")
    logging.info(f"批次大小: {batch_size} 调整后的批次大小: {adjusted_batch_size}")
    
    # 计算类别权重，用于损失函数
    class_weights = calculate_class_weights(all_labels)
    weighted_loss_fn = nn.CrossEntropyLoss(weight=class_weights.to(device))
    logging.info(f"使用带权重的损失函数，权重: {class_weights.numpy()}")
    
    scheduler_kwargs['verbose'] = True
    
    # 使用分层K折训练
    for fold, (train_idx, val_idx) in enumerate(skf.split(all_data, all_labels), 1):
        fold_start_time = time.time()
        logging.info(f"开始训练第 {fold}/{num_folds} 折")
        
        # 准备数据加载器
        train_data = Subset(dataset, train_idx)
        val_data = Subset(dataset, val_idx)
        
        # 记录当前折叠的类别分布
        train_labels = dataset.tensors[1][train_idx].numpy()
        val_labels = dataset.tensors[1][val_idx].numpy()
        train_class_counts = np.bincount(train_labels)
        val_class_counts = np.bincount(val_labels)
        
        logging.info(f"训练集类别分布: 类别0: {train_class_counts[0]}, 类别1: {train_class_counts[1]}")
        logging.info(f"验证集类别分布: 类别0: {val_class_counts[0]}, 类别1: {val_class_counts[1]}")
        
        # 创建数据加载器
        train_loader = create_data_loaders_balanced(
            train_data, batch_size, num_gpus, is_train=True
        )
        
        val_loader = create_data_loaders(
            val_data, batch_size, num_gpus
        )
        
        # 初始化模型
        model = model_class(
            input_size=input_size,
            hidden_sizes=hidden_sizes,  # 使用修改后的参数名
            output_size=output_size,
            dropout_prob=dropout_prob,
            num_heads=num_heads,
            dim_feedforward=dim_feedforward,
            num_encoder_layers=num_encoder_layers
        ).to(device)
        
        if device_ids and len(device_ids) > 1:
            model = nn.DataParallel(model, device_ids=device_ids)
        
        optimizer = optimizer_class(model.parameters(), **optimizer_kwargs)
        scheduler = scheduler_class(optimizer, **scheduler_kwargs)
        
        epoch_logs = []
        try:
            # 训练单折模型，使用带权重的损失函数
            (train_loss, val_loss, 
             train_acc, val_acc,
             train_probs, train_labels,
             val_probs, val_labels,
             best_model_state) = train_eval_transformer(
                model=model,
                train_loader=train_loader,
                val_loader=val_loader,
                loss_fn=weighted_loss_fn,  # 使用带权重的损失函数
                optimizer=optimizer,
                scheduler=scheduler,
                epochs=epochs,
                device=device,
                log_epoch=epoch_logs,
                max_grad_norm=max_grad_norm,
            )
            
            # 保存模型和相关数据
            base_filename = os.path.join(output_dir, f"{model_name}_{window_name}_fold_{fold}")
            
            # 1. 保存模型权重
            model_path = f"{base_filename}.pth"
            torch.save(best_model_state, model_path)
            logging.info(f"已保存模型权重到: {model_path}")
            
            # 2. 保存训练日志
            log_path = f"{base_filename}_epoch_logs.txt"
            with open(log_path, 'w', encoding='utf-8') as f:
                f.write('\n'.join(epoch_logs))
            logging.info(f"已保存训练日志到: {log_path}")
            
            # 3. 保存训练集预测概率
            train_probs_path = f"{base_filename}_train_probs.npy"
            np.save(train_probs_path, train_probs)
            logging.info(f"已保存训练集预测概率到: {train_probs_path}")
            
            # 4. 保存训练集标签
            train_labels_path = f"{base_filename}_train_labels.npy"
            np.save(train_labels_path, train_labels)
            logging.info(f"已保存训练集标签到: {train_labels_path}")
            
            # 5. 保存测试集预测概率
            test_probs_path = f"{base_filename}_test_probs.npy"
            np.save(test_probs_path, val_probs)
            logging.info(f"已保存测试集预测概率到: {test_probs_path}")
            
            # 6. 保存测试集标签
            test_labels_path = f"{base_filename}_test_labels.npy"
            np.save(test_labels_path, val_labels)
            logging.info(f"已保存测试集标签到: {test_labels_path}")
            
            # 计算预测值
            train_preds = np.argmax(train_probs, axis=1)
            val_preds = np.argmax(val_probs, axis=1)
            
            # 计算各种评估指标，使用zero_division=1来避免警告
            # F1分数
            train_f1 = f1_score(train_labels, train_preds, average='weighted')
            val_f1 = f1_score(val_labels, val_preds, average='weighted')
            
            # 精确率 (Precision)
            train_precision = precision_score(train_labels, train_preds, average='weighted', zero_division=1)
            val_precision = precision_score(val_labels, val_preds, average='weighted', zero_division=1)
            
            # 召回率 (Recall)
            train_recall = recall_score(train_labels, train_preds, average='weighted', zero_division=1)
            val_recall = recall_score(val_labels, val_preds, average='weighted', zero_division=1)
            
            # 特异性 (Specificity) - 需要计算混淆矩阵
            train_cm = confusion_matrix(train_labels, train_preds)
            val_cm = confusion_matrix(val_labels, val_preds)
            
            # 对于二分类，特异性 = TN / (TN + FP)
            train_tn = train_cm[0, 0]
            train_fp = train_cm[0, 1]
            train_specificity = train_tn / (train_tn + train_fp) if (train_tn + train_fp) > 0 else 0
            
            val_tn = val_cm[0, 0]
            val_fp = val_cm[0, 1]
            val_specificity = val_tn / (val_tn + val_fp) if (val_tn + val_fp) > 0 else 0
            
            # MCC (Matthews Correlation Coefficient)
            train_mcc = matthews_corrcoef(train_labels, train_preds)
            val_mcc = matthews_corrcoef(val_labels, val_preds)
            
            # 归一化MCC到[0,1]区间
            train_norm_mcc = (train_mcc + 1) / 2
            val_norm_mcc = (val_mcc + 1) / 2
            
            # 计算每个类别的性能指标
            class_counts = np.bincount(val_labels.astype(int))
            class_correct = np.bincount(val_labels.astype(int) * (val_preds == val_labels), minlength=2)
            class_acc = class_correct / class_counts
            
            logging.info(f"类别准确率: 类别0: {class_acc[0]:.4f}, 类别1: {class_acc[1]:.4f}")
            logging.info(f"F1分数: 训练: {train_f1:.4f}, 验证: {val_f1:.4f}")
            logging.info(f"精确率: 训练: {train_precision:.4f}, 验证: {val_precision:.4f}")
            logging.info(f"召回率: 训练: {train_recall:.4f}, 验证: {val_recall:.4f}")
            logging.info(f"特异性: 训练: {train_specificity:.4f}, 验证: {val_specificity:.4f}")
            logging.info(f"MCC: 训练: {train_mcc:.4f}, 验证: {val_mcc:.4f}")
            logging.info(f"归一化MCC: 训练: {train_norm_mcc:.4f}, 验证: {val_norm_mcc:.4f}")
            
            # 收集每折的结果
            fold_result = {
                'fold': fold,
                'train_loss': train_loss,
                'val_loss': val_loss,
                'train_acc': train_acc,
                'val_acc': val_acc,
                'train_f1': train_f1,
                'val_f1': val_f1,
                'train_precision': train_precision,
                'val_precision': val_precision,
                'train_recall': train_recall,
                'val_recall': val_recall,
                'train_specificity': train_specificity,
                'val_specificity': val_specificity,
                'train_mcc': train_mcc,
                'val_mcc': val_mcc,
                'train_norm_mcc': train_norm_mcc,
                'val_norm_mcc': val_norm_mcc,
                'class_acc': class_acc.tolist(),  # 添加每个类别的准确率
                'model_path': model_path,
                'train_probs_path': train_probs_path,
                'train_labels_path': train_labels_path,
                'test_probs_path': test_probs_path,
                'test_labels_path': test_labels_path,
                'log_path': log_path,
                'time_taken': time.time() - fold_start_time
            }
            results_dict['fold_results'].append(fold_result)
            
            # 更新汇总指标
            results_dict['train_losses'].append(train_loss)
            results_dict['val_losses'].append(val_loss)
            results_dict['train_accs'].append(train_acc)
            results_dict['val_accs'].append(val_acc)
            results_dict['train_f1s'].append(train_f1)
            results_dict['val_f1s'].append(val_f1)
            results_dict['train_precisions'].append(train_precision)
            results_dict['val_precisions'].append(val_precision)
            results_dict['train_recalls'].append(train_recall)
            results_dict['val_recalls'].append(val_recall)
            results_dict['train_specificities'].append(train_specificity)
            results_dict['val_specificities'].append(val_specificity)
            results_dict['train_mccs'].append(train_mcc)
            results_dict['val_mccs'].append(val_mcc)
            results_dict['train_norm_mccs'].append(train_norm_mcc)
            results_dict['val_norm_mccs'].append(val_norm_mcc)
            
            logging.info(f"第 {fold} 折训练完成")
            
        except Exception as e:
            logging.error(f"训练第 {fold} 折时出错: {str(e)}")
            logging.error("详细错误信息:", exc_info=True)
            continue
        
        logging.info(f"第 {fold} 折训练结束")
    
    # 计算总训练时间
    total_duration = time.time() - total_start_time
    hours = int(total_duration // 3600)
    minutes = int((total_duration % 3600) // 60)
    seconds = int(total_duration % 60)
    
    # 计算统计结果
    train_acc_mean = np.mean(results_dict['train_accs'])
    train_acc_std = np.std(results_dict['train_accs'])
    val_acc_mean = np.mean(results_dict['val_accs'])
    val_acc_std = np.std(results_dict['val_accs'])
    
    train_f1_mean = np.mean(results_dict['train_f1s'])
    val_f1_mean = np.mean(results_dict['val_f1s'])
    
    train_precision_mean = np.mean(results_dict['train_precisions'])
    val_precision_mean = np.mean(results_dict['val_precisions'])
    
    train_recall_mean = np.mean(results_dict['train_recalls'])
    val_recall_mean = np.mean(results_dict['val_recalls'])
    
    train_specificity_mean = np.mean(results_dict['train_specificities'])
    val_specificity_mean = np.mean(results_dict['val_specificities'])
    
    train_mcc_mean = np.mean(results_dict['train_mccs'])
    val_mcc_mean = np.mean(results_dict['val_mccs'])
    
    train_norm_mcc_mean = np.mean(results_dict['train_norm_mccs'])
    val_norm_mcc_mean = np.mean(results_dict['val_norm_mccs'])
    
    # 计算每个类别的平均准确率
    class0_accs = [res['class_acc'][0] for res in results_dict['fold_results']]
    class1_accs = [res['class_acc'][1] for res in results_dict['fold_results']]
    class0_acc_mean = np.mean(class0_accs)
    class1_acc_mean = np.mean(class1_accs)
    
    # 保存训练配置和结果，包括类别平衡信息
    config = {
        'model_name': model_name,
        'window_name': window_name,
        'model_params': {
            'input_size': input_size,
            'hidden_sizes': hidden_sizes,
            'output_size': output_size,
            'dropout_prob': dropout_prob,
            'num_heads': num_heads,
            'dim_feedforward': dim_feedforward,
            'num_encoder_layers': num_encoder_layers
        },
        'training_params': {
            'optimizer': optimizer_class.__name__,
            'optimizer_kwargs': optimizer_kwargs,
            'scheduler': scheduler_class.__name__,
            'scheduler_kwargs': scheduler_kwargs,
            'epochs': epochs,
            'batch_size': batch_size,
            'class_weights': class_weights.tolist(),  # 保存类别权重
            'balanced_sampling': True  # 标记使用了平衡采样
        },
        'results': {
            'train_acc_mean': float(train_acc_mean),
            'train_acc_std': float(train_acc_std),
            'val_acc_mean': float(val_acc_mean),
            'val_acc_std': float(val_acc_std),
            'train_f1_mean': float(train_f1_mean),
            'val_f1_mean': float(val_f1_mean),
            'train_precision_mean': float(train_precision_mean),  # 添加精确率
            'val_precision_mean': float(val_precision_mean),      # 添加精确率
            'train_recall_mean': float(train_recall_mean),        # 添加召回率
            'val_recall_mean': float(val_recall_mean),            # 添加召回率
            'train_specificity_mean': float(train_specificity_mean),  # 添加特异性
            'val_specificity_mean': float(val_specificity_mean),      # 添加特异性
            'train_mcc_mean': float(train_mcc_mean),              # 添加MCC
            'val_mcc_mean': float(val_mcc_mean),                  # 添加MCC
            'train_norm_mcc_mean': float(train_norm_mcc_mean),    # 添加归一化MCC
            'val_norm_mcc_mean': float(val_norm_mcc_mean),        # 添加归一化MCC
            'class0_acc_mean': float(class0_acc_mean),
            'class1_acc_mean': float(class1_acc_mean),
            'training_time': {
                'hours': hours,
                'minutes': minutes,
                'seconds': seconds
            },
            'fold_results': results_dict['fold_results']
        }
    }
    
    config_path = os.path.join(output_dir, f"{model_name}_{window_name}_config.json")
    with open(config_path, 'w', encoding='utf-8') as f:
        json.dump(config, f, indent=4, ensure_ascii=False)
    
    # 记录训练总结
    logging.info("\n训练总结:")
    logging.info(f"- 总训练时间: {hours}小时 {minutes}分钟 {seconds}秒")
    logging.info(f"- 平均训练准确率: {train_acc_mean:.4f} ± {train_acc_std:.4f}")
    logging.info(f"- 平均验证准确率: {val_acc_mean:.4f} ± {val_acc_std:.4f}")
    logging.info(f"- 平均训练F1分数: {train_f1_mean:.4f}")
    logging.info(f"- 平均验证F1分数: {val_f1_mean:.4f}")
    logging.info(f"- 平均训练精确率: {train_precision_mean:.4f}")
    logging.info(f"- 平均验证精确率: {val_precision_mean:.4f}")
    logging.info(f"- 平均训练召回率: {train_recall_mean:.4f}")
    logging.info(f"- 平均验证召回率: {val_recall_mean:.4f}")
    logging.info(f"- 平均训练特异性: {train_specificity_mean:.4f}")
    logging.info(f"- 平均验证特异性: {val_specificity_mean:.4f}")
    logging.info(f"- 平均训练MCC: {train_mcc_mean:.4f}")
    logging.info(f"- 平均验证MCC: {val_mcc_mean:.4f}")
    logging.info(f"- 平均训练归一化MCC: {train_norm_mcc_mean:.4f}")
    logging.info(f"- 平均验证归一化MCC: {val_norm_mcc_mean:.4f}")
    logging.info(f"- 类别0平均准确率: {class0_acc_mean:.4f}")
    logging.info(f"- 类别1平均准确率: {class1_acc_mean:.4f}")
    logging.info(f"- 平均验证损失: {np.mean(results_dict['val_losses']):.4f} ± {np.std(results_dict['val_losses']):.4f}")
    
    return results_dict

In [20]:
import torch
print("CUDA 是否可用:", torch.cuda.is_available())
print("当前 PyTorch 版本:", torch.__version__)
print("CUDA 版本:", torch.version.cuda)
print("GPU 数量:", torch.cuda.device_count())

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

CUDA 是否可用: True
当前 PyTorch 版本: 2.5.1+cu118
CUDA 版本: 11.8
GPU 数量: 2
GPU 0: NVIDIA GeForce RTX 4090
GPU 1: NVIDIA GeForce RTX 4090


检查数据集的大小，以便更科学的数据平衡

In [21]:
import os
import numpy as np

base_path = r"C:\Users\USTC\Desktop\地磁论文（5级以上）"

# 检查各个时间窗口的数据分布
window_periods = ['7', '14', '30']

for period in window_periods:
    window_path = os.path.join(base_path, f"window-{period}")
    
    try:
        # 加载数据文件
        data_0_path = os.path.join(window_path, "data_0.npy")
        data_1_path = os.path.join(window_path, "data_1.npy")
        
        if os.path.exists(data_0_path) and os.path.exists(data_1_path):
            data_0 = np.load(data_0_path)
            data_1 = np.load(data_1_path)
            
            # 提取标签信息
            labels_0 = data_0[:, -1].astype(int)
            labels_1 = data_1[:, -1].astype(int)
            
            # 合并数据和标签
            all_data = np.concatenate([data_0, data_1], axis=0)
            all_labels = all_data[:, -1].astype(int)
            
            # 计算类别分布
            class_counts = np.bincount(all_labels)
            
            print(f"\n=== 时间窗口: {period}天 ===")
            print(f"数据形状 - data_0: {data_0.shape}, data_1: {data_1.shape}")
            print(f"总样本数: {len(all_labels)}")
            if len(class_counts) >= 2:
                print(f"类别0样本数: {class_counts[0]}")
                print(f"类别1样本数: {class_counts[1]}")
                print(f"类别比例(1:0): 1:{class_counts[0]/class_counts[1]:.2f}")
            else:
                print(f"警告: 只有一个类别 - {class_counts}")
                
            # 计算平衡建议
            if class_counts[0] > class_counts[1]:
                minority_class = 1
                minority_count = class_counts[1]
                majority_count = class_counts[0]
            else:
                minority_class = 0
                minority_count = class_counts[0]
                majority_count = class_counts[1]
                
            imbalance_ratio = majority_count / minority_count
            
            print(f"\n数据不平衡分析:")
            print(f"不平衡比例: {imbalance_ratio:.2f}:1")
            
            if imbalance_ratio < 1.5:
                print("评估: 数据集相对平衡，简单的类别权重调整应该足够")
                weight_suggestion = f"类别权重建议: [1.0, {imbalance_ratio:.2f}]" if minority_class == 1 else f"类别权重建议: [{imbalance_ratio:.2f}, 1.0]"
                print(weight_suggestion)
            elif imbalance_ratio < 10:
                print("评估: 中度不平衡，建议结合类别权重和采样技术")
                # 计算平方根缩放的权重
                sqrt_weights = np.sqrt(class_counts) / np.sum(np.sqrt(class_counts)) * len(class_counts)
                weights_str = "[{:.2f}, {:.2f}]".format(sqrt_weights[0], sqrt_weights[1])
                print(f"类别权重建议(平方根缩放): {weights_str}")
                print("采样策略: 使用WeightedRandomSampler进行训练")
            else:
                print("评估: 严重不平衡，建议使用组合策略")
                print("建议策略:")
                print("1. 使用有效样本数计算权重")
                print("2. 考虑使用Focal Loss代替CrossEntropyLoss")
                print("3. 结合过采样和欠采样技术")
                print("4. 使用数据增强生成少数类样本")
        else:
            print(f"错误: 无法找到窗口 {period} 的数据文件")
    except Exception as e:
        print(f"处理窗口 {period} 时出错: {str(e)}")


=== 时间窗口: 7天 ===
数据形状 - data_0: (1661, 10081), data_1: (999, 10081)
总样本数: 2660
类别0样本数: 1661
类别1样本数: 999
类别比例(1:0): 1:1.66

数据不平衡分析:
不平衡比例: 1.66:1
评估: 中度不平衡，建议结合类别权重和采样技术
类别权重建议(平方根缩放): [1.13, 0.87]
采样策略: 使用WeightedRandomSampler进行训练

=== 时间窗口: 14天 ===
数据形状 - data_0: (1878, 20161), data_1: (826, 20161)
总样本数: 2704
类别0样本数: 1878
类别1样本数: 826
类别比例(1:0): 1:2.27

数据不平衡分析:
不平衡比例: 2.27:1
评估: 中度不平衡，建议结合类别权重和采样技术
类别权重建议(平方根缩放): [1.20, 0.80]
采样策略: 使用WeightedRandomSampler进行训练

=== 时间窗口: 30天 ===
数据形状 - data_0: (1421, 43201), data_1: (770, 43201)
总样本数: 2191
类别0样本数: 1421
类别1样本数: 770
类别比例(1:0): 1:1.85

数据不平衡分析:
不平衡比例: 1.85:1
评估: 中度不平衡，建议结合类别权重和采样技术
类别权重建议(平方根缩放): [1.15, 0.85]
采样策略: 使用WeightedRandomSampler进行训练


In [22]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset, WeightedRandomSampler
from torch.optim import lr_scheduler
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import f1_score
import logging
import time
import json
import warnings
from typing import Dict, Any, Optional, List, Tuple
import copy

# 忽略NCCL警告
warnings.filterwarnings("ignore", message="PyTorch is not compiled with NCCL support")
import warnings
warnings.filterwarnings("ignore")

def setup_logging(log_file: str = 'training.log'):
    """设置日志配置"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'  # 移除了 ,%f，因为这不是标准的时间格式
    )
    # 如果需要添加文件处理器
    file_handler = logging.FileHandler(log_file, encoding='utf-8')
    file_handler.setFormatter(
        logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    )
    logger = logging.getLogger(__name__)
    logger.addHandler(file_handler)
    return logger

def setup_device() -> Tuple[torch.device, Optional[List[int]]]:
    """配置训练设备，最大化利用所有可用NVIDIA GPU"""
    if torch.cuda.device_count() > 1:
        # 获取所有可用GPU
        device_count = torch.cuda.device_count()
        device_ids = list(range(device_count))
        device = torch.device(f'cuda:{device_ids[0]}')
        
        # 输出每个GPU的信息
        gpu_info = []
        for i in device_ids:
            gpu_name = torch.cuda.get_device_name(i)
            gpu_info.append(f"GPU {i}: {gpu_name}")
        
        logging.info(f"使用 {len(device_ids)} 个GPU设备: {device_ids}")
        logging.info("GPU详情: " + ", ".join(gpu_info))
        
        # 设置主设备，确保初始化顺序正确
        torch.cuda.set_device(device_ids[0])
        
        return device, device_ids
    elif torch.cuda.is_available():
        device = torch.device('cuda:0')
        gpu_name = torch.cuda.get_device_name(0)
        logging.info(f"使用单个GPU设备: {gpu_name}")
        return device, None
    else:
        device = torch.device('cpu')
        logging.info("使用CPU设备")
        return device, None

def calculate_class_weights(labels):
    """计算平方根缩放的类别权重"""
    class_counts = np.bincount(labels)
    logging.info(f"类别分布: 类别0: {class_counts[0]}, 类别1: {class_counts[1]}")
    
    # 使用平方根缩放计算权重
    weights_sqrt = np.sqrt(class_counts.sum() / class_counts / len(class_counts))
    return torch.FloatTensor(weights_sqrt)

def create_balanced_sampler(dataset):
    """创建平衡采样器"""
    # 提取数据集的标签
    if isinstance(dataset, TensorDataset):
        labels = dataset.tensors[1].numpy()
    elif isinstance(dataset, Subset):
        labels = dataset.dataset.tensors[1][dataset.indices].numpy()
    else:
        raise TypeError("不支持的数据集类型")
    
    # 计算类别权重（反比例，但不使用平方根缩放）
    class_counts = np.bincount(labels)
    logging.info(f"原始类别分布: {class_counts}")
    
    # 计算每个样本的权重
    class_weights = 1.0 / class_counts
    # 确保少数类有更高的采样概率
    sample_weights = class_weights[labels]
    
    # 创建采样器
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True  # 允许重复采样
    )
    
    return sampler

def create_data_loaders_balanced(dataset, batch_size, num_gpus=1, is_train=True):
    """创建带有平衡采样的数据加载器"""
    effective_batch_size = batch_size * num_gpus if num_gpus > 1 else batch_size
    
    if is_train:
        # 训练集使用加权采样
        sampler = create_balanced_sampler(dataset)
        logging.info(f"批次大小: {batch_size} 调整后的批次大小: {effective_batch_size} (使用平衡采样)")
        return DataLoader(
            dataset,
            batch_size=effective_batch_size,
            sampler=sampler,  # 使用采样器替代shuffle
            num_workers=4 * num_gpus,
            pin_memory=True
        )
    else:
        # 验证集不需要平衡采样
        logging.info(f"批次大小: {batch_size} 调整后的批次大小: {effective_batch_size}")
        return DataLoader(
            dataset,
            batch_size=effective_batch_size,
            shuffle=False,
            num_workers=4 * num_gpus,
            pin_memory=True
        )

def create_data_loaders(dataset: TensorDataset, batch_size: int, num_gpus: int = 1) -> DataLoader:
    """创建数据加载器"""
    effective_batch_size = batch_size * num_gpus if num_gpus > 1 else batch_size
    logging.info(f"批次大小: {batch_size} 调整后的批次大小: {effective_batch_size}")
    
    return DataLoader(
        dataset,
        batch_size=effective_batch_size,
        shuffle=True,
        num_workers=4 * num_gpus,
        pin_memory=True
    )

def create_model(model_class, input_size, hidden_sizes, output_size, dropout_prob, device, device_ids=None):
    """创建模型"""
    model = model_class(
        input_size=input_size,
        hidden_sizes=hidden_sizes,
        output_size=output_size,
        dropout_prob=dropout_prob
    )
    
    if device_ids and len(device_ids) > 1:
        model = nn.DataParallel(model, device_ids=device_ids)
    
    return model.to(device)

def reshape_data_for_transformer(X: np.ndarray, seq_length: int) -> np.ndarray:
    """重塑数据为Transformer输入格式"""
    try:
        batch_size = X.shape[0]
        feature_per_timestamp = X.shape[1] // seq_length
        X_reshaped = X.reshape(batch_size, seq_length, feature_per_timestamp)
        logging.info(f"数据重塑成功: {X.shape} -> {X_reshaped.shape}")
        return X_reshaped
    except Exception as e:
        logging.error(f"数据重塑失败: {str(e)}")
        raise

def main_transformer(optimize: bool = False) -> None:
    """主训练函数 - 使用分层K折交叉验证和F1分数优化"""
    setup_logging()  # 设置日志
    device, device_ids = setup_device()
    base_path = r"C:\Users\USTC\Desktop\地磁论文（5级以上）"
    output_dir = os.path.join(base_path, '对比模型结果', 'Transformer模型结果')
    os.makedirs(output_dir, exist_ok=True)

    logging.info(f"开始训练流程 - 使用设备: {device}")

    window_mapping = {
        "window-7": {"name": "7天", "seq_length": 7},
        "window-14": {"name": "14天", "seq_length": 14},
        "window-30": {"name": "30天", "seq_length": 30}
    }

    try:
        # 训练参数设置
        training_config = {
            'hidden_sizes': [512, 256],    # 隐藏层大小
            'num_layers': 2,               # 层数保持为2
            'dropout_prob': 0.3,           # dropout率
            'learning_rate': 0.001,        # 学习率
            'batch_size': 32,              # batch size
            'weight_decay': 0.01,          # L2正则化参数
            'max_grad_norm': 1.0,          # 梯度裁剪阈值
            'num_heads': 8,               # Transformer特有：注意力头数量
            'dim_feedforward': 2048       # Transformer特有：前馈网络维度
        }
        
        optimizer_kwargs = {
            'lr': training_config['learning_rate'],
            'weight_decay': training_config['weight_decay']
        }
        
        scheduler_kwargs = {
            'mode': 'min',
            'factor': 0.1,
            'patience': 10,
            'min_lr': 1e-6,
            'verbose': True
        }
        
        num_gpus = len(device_ids) if device_ids else 1
        
        for window_period in ['7', '14', '30']:
            current_window_name = f"window-{window_period}"
            window_info = window_mapping[current_window_name]
            
            logging.info(f"\n=== 开始处理时间窗口: {window_info['name']} ===")

            try:
                # 加载数据
                data_dir = os.path.join(base_path, current_window_name)
                data_0 = np.load(os.path.join(data_dir, "data_0.npy"))
                data_1 = np.load(os.path.join(data_dir, "data_1.npy"))
                
                data = np.concatenate([data_0, data_1], axis=0)
                X = data[:, :-1]
                Y = data[:, -1].astype(int)
                
                # 记录类别分布
                class_counts = np.bincount(Y)
                logging.info(f"类别分布: 类别0: {class_counts[0]}, 类别1: {class_counts[1]}")
                
                # 计算类别权重，用于损失函数
                class_weights = calculate_class_weights(Y)
                logging.info(f"使用平方根缩放的类别权重: {class_weights}")
                weighted_loss = nn.CrossEntropyLoss(weight=class_weights)
                
                # 重塑数据为Transformer格式
                X = reshape_data_for_transformer(X, window_info['seq_length'])
                
                # 转换为张量
                X_tensor = torch.from_numpy(X).type(torch.float32)
                Y_tensor = torch.from_numpy(Y).type(torch.long)
                dataset = TensorDataset(X_tensor, Y_tensor)

                # 使用分层K折交叉验证训练
                results = kfold_train_eval_transformer(
                    model_class=TransformerModel,
                    dataset=dataset,
                    loss_fn=weighted_loss,  # 使用带权重的损失函数
                    optimizer_class=optim.AdamW,
                    optimizer_kwargs=optimizer_kwargs,
                    scheduler_class=optim.lr_scheduler.ReduceLROnPlateau,
                    scheduler_kwargs=scheduler_kwargs,
                    epochs=100,
                    device=device,
                    device_ids=device_ids,
                    num_folds=5,
                    output_dir=output_dir,
                    model_name="TransformerModel",
                    window_name=window_info['name'],
                    mapped_window_name=current_window_name,
                    input_size=X.shape[2],
                    output_size=2,
                    hidden_sizes=[training_config['dim_feedforward'], training_config['dim_feedforward'] // 2],  # 将Transformer参数映射到通用参数
                    dropout_prob=training_config['dropout_prob'],
                    batch_size=training_config['batch_size'],
                    max_grad_norm=training_config['max_grad_norm']
                )

                # 执行基于F1分数的模型集成
                perform_model_ensemble(
                    model_name="TransformerModel",
                    window_name=window_info['name'],
                    output_dir=output_dir,
                    device=device,
                    device_ids=device_ids,
                    model_class=TransformerModel,
                    input_size=X.shape[2],
                    hidden_sizes=training_config['hidden_sizes'],
                    output_size=2,
                    dropout_prob=training_config['dropout_prob'],
                    num_folds=5
                )

            except Exception as e:
                logging.error(f"处理 {window_info['name']} 时出错: {str(e)}")
                logging.error("详细错误信息:", exc_info=True)
                continue

        logging.info("\n=== 所有处理完成！===")

    except Exception as e:
        logging.error(f"程序执行出错: {str(e)}")
        raise

def perform_model_ensemble(
    model_name: str,
    window_name: str,
    output_dir: str,
    device: torch.device,
    device_ids: Optional[List[int]],
    model_class: Any,
    input_size: int,
    hidden_sizes: List[int],
    output_size: int,
    dropout_prob: float,
    num_folds: int
) -> None:
    """
    执行模型集成过程，使用F1分数优化权重分配，同时保存全面的评估指标
    
    Args:
        model_name: 模型名称
        window_name: 时间窗口名称
        output_dir: 输出目录
        device: 设备
        device_ids: GPU设备ID列表
        model_class: 模型类
        input_size: 输入维度
        hidden_sizes: Transformer隐藏层大小列表
        output_size: 输出维度
        dropout_prob: Dropout概率
        num_folds: 折叠数量
    """
    from sklearn.metrics import f1_score, precision_score, recall_score, matthews_corrcoef, confusion_matrix
    
    logging.info(f"开始进行模型集成 - {model_name}_{window_name}")
    
    try:
        # 加载各个折叠的模型
        models = []
        model_metrics = {
            'f1_scores': [],            # F1分数
            'precisions': [],           # 精确率
            'recalls': [],              # 召回率
            'specificities': [],        # 特异性
            'mccs': [],                 # MCC
            'norm_mccs': []             # 归一化MCC
        }
        
        for fold in range(1, num_folds + 1):
            # 创建基础模型
            model = model_class(
                input_size=input_size,
                hidden_sizes=hidden_sizes,
                output_size=output_size,
                dropout_prob=dropout_prob
            ).to(device)
            
            if device_ids and len(device_ids) > 1:
                model = nn.DataParallel(model, device_ids=device_ids)
            
            # 加载模型权重
            model_path = os.path.join(output_dir, f"{model_name}_{window_name}_fold_{fold}.pth")
            if os.path.exists(model_path):
                state_dict = torch.load(model_path)
                if device_ids and len(device_ids) > 1:
                    if not list(state_dict.keys())[0].startswith('module.'):
                        state_dict = {'module.' + k: v for k, v in state_dict.items()}
                model.load_state_dict(state_dict)
                model.eval()
                models.append(model)
                
                # 加载验证数据，计算各项指标
                test_probs_path = os.path.join(output_dir, f"{model_name}_{window_name}_fold_{fold}_test_probs.npy")
                test_labels_path = os.path.join(output_dir, f"{model_name}_{window_name}_fold_{fold}_test_labels.npy")
                
                if os.path.exists(test_probs_path) and os.path.exists(test_labels_path):
                    test_probs = np.load(test_probs_path)
                    test_labels = np.load(test_labels_path)
                    
                    test_preds = np.argmax(test_probs, axis=1)
                    
                    # 计算所有评估指标
                    fold_f1 = f1_score(test_labels, test_preds, average='weighted')
                    fold_precision = precision_score(test_labels, test_preds, average='weighted')
                    fold_recall = recall_score(test_labels, test_preds, average='weighted')
                    
                    # 计算特异性 (针对二分类)
                    fold_cm = confusion_matrix(test_labels, test_preds)
                    fold_tn = fold_cm[0, 0]
                    fold_fp = fold_cm[0, 1]
                    fold_specificity = fold_tn / (fold_tn + fold_fp) if (fold_tn + fold_fp) > 0 else 0
                    
                    # 计算MCC和归一化MCC
                    fold_mcc = matthews_corrcoef(test_labels, test_preds)
                    fold_norm_mcc = (fold_mcc + 1) / 2  # 归一化到[0,1]区间
                    
                    # 确保F1分数非零(用于权重计算)
                    fold_f1 = max(fold_f1, 0.01)
                    
                    # 存储所有指标
                    model_metrics['f1_scores'].append(fold_f1)
                    model_metrics['precisions'].append(fold_precision)
                    model_metrics['recalls'].append(fold_recall)
                    model_metrics['specificities'].append(fold_specificity)
                    model_metrics['mccs'].append(fold_mcc)
                    model_metrics['norm_mccs'].append(fold_norm_mcc)
                    
                    logging.info(f"第 {fold} 折模型评估指标:")
                    logging.info(f"  F1分数: {fold_f1:.4f}")
                    logging.info(f"  精确率: {fold_precision:.4f}")
                    logging.info(f"  召回率: {fold_recall:.4f}")
                    logging.info(f"  特异性: {fold_specificity:.4f}")
                    logging.info(f"  MCC: {fold_mcc:.4f}")
                    logging.info(f"  归一化MCC: {fold_norm_mcc:.4f}")
                else:
                    # 如果找不到验证数据，从配置文件中尝试获取各项指标
                    config_path = os.path.join(output_dir, f"{model_name}_{window_name}_config.json")
                    if os.path.exists(config_path):
                        with open(config_path, 'r', encoding='utf-8') as f:
                            config = json.load(f)
                            fold_results = config['results']['fold_results']
                            fold_result = next((result for result in fold_results if result['fold'] == fold), None)
                            
                            if fold_result:
                                # 提取所有可用的指标
                                fold_f1 = fold_result.get('val_f1', 1.0)
                                model_metrics['f1_scores'].append(fold_f1)
                                
                                if 'val_precision' in fold_result:
                                    model_metrics['precisions'].append(fold_result['val_precision'])
                                else:
                                    model_metrics['precisions'].append(0.0)
                                    
                                if 'val_recall' in fold_result:
                                    model_metrics['recalls'].append(fold_result['val_recall'])
                                else:
                                    model_metrics['recalls'].append(0.0)
                                    
                                if 'val_specificity' in fold_result:
                                    model_metrics['specificities'].append(fold_result['val_specificity'])
                                else:
                                    model_metrics['specificities'].append(0.0)
                                    
                                if 'val_mcc' in fold_result:
                                    model_metrics['mccs'].append(fold_result['val_mcc'])
                                else:
                                    model_metrics['mccs'].append(0.0)
                                    
                                if 'val_norm_mcc' in fold_result:
                                    model_metrics['norm_mccs'].append(fold_result['val_norm_mcc'])
                                else:
                                    model_metrics['norm_mccs'].append(0.5)  # 0.5是归一化MCC的中间值
                                
                                logging.info(f"第 {fold} 折模型F1分数(从配置获取): {fold_f1:.4f}")
                            else:
                                # 如果找不到特定折的结果，使用默认值
                                model_metrics['f1_scores'].append(1.0)
                                model_metrics['precisions'].append(0.0)
                                model_metrics['recalls'].append(0.0)
                                model_metrics['specificities'].append(0.0)
                                model_metrics['mccs'].append(0.0)
                                model_metrics['norm_mccs'].append(0.5)
                                logging.warning(f"未找到第 {fold} 折的F1分数，使用默认值1.0")
                    else:
                        # 如果找不到配置文件，使用默认值
                        model_metrics['f1_scores'].append(1.0)
                        model_metrics['precisions'].append(0.0)
                        model_metrics['recalls'].append(0.0)
                        model_metrics['specificities'].append(0.0)
                        model_metrics['mccs'].append(0.0)
                        model_metrics['norm_mccs'].append(0.5)
                        logging.warning(f"未找到第 {fold} 折的验证集结果，使用默认F1=1.0")
                
                logging.info(f"已加载第 {fold} 折模型")
            else:
                logging.warning(f"未找到第 {fold} 折模型权重文件: {model_path}")
        
        if models:
            # 使用基于F1分数的集成权重
            weights = np.array(model_metrics['f1_scores'])
            weights = weights / weights.sum()
            logging.info(f"基于F1分数的模型权重: {weights}")
            
            # 创建集成模型
            ensemble_model = EnsembleModel(
                base_models=models,
                ensemble_weights=weights
            ).to(device)
            
            if device_ids and len(device_ids) > 1:
                ensemble_model = nn.DataParallel(ensemble_model, device_ids=device_ids)
            
            # 保存集成模型
            ensemble_path = os.path.join(output_dir, f"{model_name}_{window_name}.pth")
            torch.save(ensemble_model.state_dict(), ensemble_path)
            
            # 计算各项指标的平均值
            avg_metrics = {}
            for metric_name, values in model_metrics.items():
                if values:
                    avg_metrics[f'avg_{metric_name}'] = float(np.mean(values))
                    avg_metrics[f'std_{metric_name}'] = float(np.std(values))
            
            # 保存集成配置，包含所有评估指标
            ensemble_config = {
                'model_name': model_name,
                'window_name': window_name,
                'ensemble_weights': weights.tolist(),
                'ensemble_method': 'f1_weighted',  # 标记使用F1加权
                'model_params': {
                    'input_size': input_size,
                    'hidden_sizes': hidden_sizes,
                    'output_size': output_size,
                    'dropout_prob': dropout_prob
                },
                'fold_metrics': {
                    'f1_scores': model_metrics['f1_scores'],
                    'precisions': model_metrics['precisions'],
                    'recalls': model_metrics['recalls'],
                    'specificities': model_metrics['specificities'],
                    'mccs': model_metrics['mccs'],
                    'norm_mccs': model_metrics['norm_mccs']
                },
                'average_metrics': avg_metrics
            }
            
            config_path = os.path.join(output_dir, f"{model_name}_{window_name}_ensemble_config.json")
            with open(config_path, 'w', encoding='utf-8') as f:
                json.dump(ensemble_config, f, indent=4, ensure_ascii=False)
            
            # 输出集成模型的平均表现
            logging.info(f"F1加权集成模型已保存: {ensemble_path}")
            logging.info("集成模型平均评估指标:")
            for metric_name, value in avg_metrics.items():
                if metric_name.startswith('avg_'):
                    display_name = metric_name[4:].replace('_', ' ')
                    logging.info(f"  平均{display_name}: {value:.4f}")
            
            logging.info(f"集成配置已保存: {config_path}")
        else:
            logging.error("没有可用的模型进行集成")
            
    except Exception as e:
        logging.error(f"模型集成过程出错: {str(e)}")
        logging.error("详细错误信息:", exc_info=True)

def load_ensemble_model(
    model_path: str,
    config_path: str,
    device: torch.device,
    model_class: Any,
    device_ids: Optional[List[int]] = None
) -> EnsembleModel:
    """
    加载集成模型
    
    Args:
        model_path: 模型路径
        config_path: 配置文件路径
        device: 设备
        model_class: 模型类
        device_ids: GPU设备ID列表
    
    Returns:
        EnsembleModel: 加载的集成模型
    """
    with open(config_path, 'r') as f:
        config = json.load(f)
    
    model_params = config['model_params']
    
    base_models = []
    num_folds = len(config['ensemble_weights'])
    
    for _ in range(num_folds):
        # 创建模型实例，处理Transformer特有参数
        model_kwargs = {
            'input_size': model_params['input_size'],
            'hidden_sizes': model_params['hidden_sizes'],
            'output_size': model_params['output_size'],
            'dropout_prob': model_params['dropout_prob']
        }
        
        # 添加Transformer特有参数如果存在
        if 'num_heads' in model_params:
            model_kwargs['num_heads'] = model_params['num_heads']
        if 'dim_feedforward' in model_params:
            model_kwargs['dim_feedforward'] = model_params['dim_feedforward']
        if 'num_encoder_layers' in model_params:
            model_kwargs['num_encoder_layers'] = model_params['num_encoder_layers']
        
        model = model_class(**model_kwargs).to(device)
        base_models.append(model)
    
    ensemble_model = EnsembleModel(
        base_models=base_models,
        ensemble_weights=config['ensemble_weights']
    ).to(device)
    
    if device_ids and len(device_ids) > 1:
        ensemble_model = nn.DataParallel(ensemble_model, device_ids=device_ids)
    
    ensemble_model.load_state_dict(torch.load(model_path))
    return ensemble_model

if __name__ == "__main__":
    main_transformer()

2025-03-11 17:07:23 - INFO - 使用 2 个GPU设备: [0, 1]
2025-03-11 17:07:23 - INFO - GPU详情: GPU 0: NVIDIA GeForce RTX 4090, GPU 1: NVIDIA GeForce RTX 4090
2025-03-11 17:07:23 - INFO - 开始训练流程 - 使用设备: cuda:0
2025-03-11 17:07:23 - INFO - 
=== 开始处理时间窗口: 7天 ===
2025-03-11 17:07:23 - INFO - 类别分布: 类别0: 1661, 类别1: 999
2025-03-11 17:07:23 - INFO - 类别分布: 类别0: 1661, 类别1: 999
2025-03-11 17:07:23 - INFO - 使用平方根缩放的类别权重: tensor([0.8948, 1.1538])
2025-03-11 17:07:23 - INFO - 数据重塑成功: (2660, 10080) -> (2660, 7, 1440)
2025-03-11 17:07:23 - INFO - 使用设备数量: 2
2025-03-11 17:07:23 - INFO - 批次大小: 32 调整后的批次大小: 64
2025-03-11 17:07:23 - INFO - 类别分布: 类别0: 1661, 类别1: 999
2025-03-11 17:07:23 - INFO - 使用带权重的损失函数，权重: [0.89483094 1.1538333 ]
2025-03-11 17:07:23 - INFO - 开始训练第 1/5 折
2025-03-11 17:07:23 - INFO - 训练集类别分布: 类别0: 1328, 类别1: 800
2025-03-11 17:07:23 - INFO - 验证集类别分布: 类别0: 333, 类别1: 199
2025-03-11 17:07:23 - INFO - 原始类别分布: [1328  800]
2025-03-11 17:07:23 - INFO - 批次大小: 32 调整后的批次大小: 64 (使用平衡采样)
2025-03-11 17:07:23 - IN